In [ ]:
# Multiple generation of random data (useful to compute the p-value)
rand_generations = 100000
mult_x = [x for i in range(rand_generations)] # 100000 batches of features
mult_y = [np.sin(x) + 0.1*np.power(x,2) + 0.5*np.random.randn(100,1)
            for i in range(rand_generations)] # 100000 batches of targets

# Recap of optimized parameters (from Pytorch)
W = model.weight.item()
b = model.bias.item()

# Compute multiple MSEs (learned model)
mult_y_hat = [mult_x[i].dot(W) + b
                for i in range(rand_generations)]

mult_loss_fit = [mult_y_hat[i] - mult_y[i]
                for i in range(rand_generations)]

mult_mse_fit = [np.sum(np.power(mult_loss_fit[i], 2)) / (2 * n)
                for i in range(rand_generations)]

# Compute multiple MSEs (mean model)
mult_y_mean = [np.mean(mult_y[i])
                for i in range(rand_generations)]

mult_loss_mean = [mult_y_mean[i] - mult_y[i]
                for i in range(rand_generations)]

mult_mse_mean = [np.sum(np.power(mult_loss_mean[i], 2)) / (2 * n)
                for i in range(rand_generations)]

# Compute multiple F
mult_f_ratio = [compute_f_ratio(mult_mse_fit[i], mult_mse_mean[i], p_fit, p_mean, n)
                for i in range(rand_generations)]

# Plot the multiple F ratios generated from random data
plt.figure(figsize=(20,25))

# Emphasize the generated F ratio among all generated random data
plt.subplot(4,2,1)
heights, bins, patches = plt.hist(x=Counter(mult_f_ratio), bins='auto',
                            color='#0504aa',
                            alpha=0.7, rwidth=0.85)

print(heights)
print(bins)

idx = (np.abs(bins - f_ratio)).argmin() # Visualization trick: closest F ratio in bins 

patches[idx].set_fc('#ff8c00')
plt.grid(axis='y', alpha=0.75)
plt.xlabel('F')
plt.ylabel('Frequency of F')
maxfreq = heights.max()
plt.ylim(ymax=np.ceil(maxfreq / 10) * 10 if maxfreq % 10 else maxfreq + 10) # Clean upper y-axis limit.

# Emphasize the all bars that have a probability equal or less than the compute F ratio
# This will show a graphical representation of the p-value
plt.subplot(4,2,2)
heights, bins, patches = plt.hist(x=mult_f_ratio, bins='auto',
                            color='#0504aa',
                            alpha=0.7, rwidth=0.85)

idx_p_value = list(np.argwhere(heights < heights[idx]).reshape(-1,))
patches[idx].set_fc('#ff8c00') # Coluring f ratio
for p in idx_p_value:
    patches[p].set_fc('#ffd700') # Coluring f < f ratio

plt.grid(axis='y', alpha=0.75)
plt.xlabel('F')
plt.ylabel('Frequency of F')
maxfreq = heights.max()
plt.ylim(ymax=np.ceil(maxfreq / 10) * 10 if maxfreq % 10 else maxfreq + 10) # Clean upper y-axis limit.

# Plot!
plt.show()

# Recap of number of samples
num_samples = np.sum(heights)

# p-value is the sum of 3 different probabilities

# probability 1
p1 = heights[idx] / num_samples # Probability of the sample

# probability 2
p2 = (heights == heights[idx]).sum() - 1 # Cases with the same probability (excluding the sample)
p2 = p2 * heights[idx]
p2 = p2 / num_samples # Cases with the same probability of the samples

# probability 3
idx_p_value = list(np.argwhere(heights < heights[idx]).reshape(-1,)) # Recap of the indices with less probability
occurrences = 0
for p in idx_p_value:
    occurrences += heights[p]
p3 = occurrences / num_samples

p_value = p1 + p2 + p3

print('----- p_value: %.4f\n' % p_value)
print()
if p_value < 0.05:  # Setting our significance level at 5%
    print('The computed R_squared is statistically relevant.')
else:
    print('The computed R_squared is not statistically relevant.')


https://openclassrooms.com/en/courses/5873596-design-effective-statistical-models-to-understand-your-data/6229141-build-and-interpret-a-univariate-linear-regression-model#:~:text=null%20hypothesis%20anyway.-,R%2DSquared,positive%20and%20lower%20than%201.

